## Part IV Decompounded Options

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import interpolate
from math import log, sqrt, exp
from scipy import integrate

### Import forward swap, discount factors

In [2]:
data_FS = pd.read_csv('forward_swap.csv', header = 0, index_col = 0)
data_FS = data_FS.rename(columns={'Expiry in year(s)':'Expiry','Tenor in year(s)':'Tenor','Par_Swap_Rate':'Swap_Rate'})

data_DF = pd.read_csv('oi_libor_disfactors.csv', header = 0, index_col = 0)
data_DF = data_DF.rename(columns={'d_l':'LIBOR_DF','tenor':'Tenor','ois_rate':'Rate','f_o':'f','d_o':'OIS_DF','irs_rate':'IRS','f_l':'Libor'})

### Get SABR parameters

In [3]:
data_alpha = pd.read_csv('SABR_alpha.csv', header = 0, index_col = 0)
data_rho = pd.read_csv('SABR_rho.csv', header = 0, index_col = 0)
data_nu = pd.read_csv('SABR_nu.csv', header = 0, index_col = 0)

In [4]:
data_alpha

,1Y,2Y,3Y,5Y,10Y
1Y,0.139070,0.184651,0.196851,0.178052,0.170749
5Y,0.166527,0.199497,0.210348,0.191091,0.177421
10Y,0.177383,0.195282,0.207103,0.201575,0.181396


In [5]:
alpha = data_alpha.loc['5Y','10Y']
rho = data_rho.loc['5Y','10Y']
nu = data_nu.loc['5Y','10Y']
# Beta is predefined in part 2, beta=0.9

### Define SABR model

In [6]:
def SABR(F, K, T, alpha, beta, rho, nu):
    
    """
    Calculate the SABR volatility for a given set of parameters and market data.
    
    Parameters:
    alpha (float): initial volatility level
    beta (float): elasticity of volatility with respect to asset price
    rho (float): correlation between asset price and volatility
    nu (float): volatility of volatility
    F (float): forward price
    K (float): strike price
    T (float): time to maturity (in years)
    
    Returns:
    float: the SABR volatility
    """
        
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*log(F/X)
        zhi = log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [7]:
# test sample inputs
alpha_ = 0.2
beta_ = 0.5
rho_ = -0.25
nu_ = 0.4
F_ = 100
K_ = 90
T_ = 1

# calculate the SABR volatility
vol = SABR(F_, K_, T_, alpha_, beta_, rho_, nu_)

# print the result
print("SABR volatility:", vol)

SABR volatility: 0.0316890806831793


### Deine Black76 model

In [8]:
from scipy.stats import norm
import math

def black76_lognormal(option_type, F, K, r, sigma, T):
    """
    Calculate the Black76 option price for a given set of market data.
    
    Parameters:
    option_type (str): 'call' or 'put'
    F (float): forward price
    K (float): strike price
    r (float): risk-free interest rate
    sigma (float): annualized volatility
    T (float): time to maturity (in years)
    
    Returns:
    float: the Black76 option price
    """
    # calculate the option price
    d1 = (math.log(F / K) + ((sigma ** 2) / 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    if option_type == 'Call':
        option_price = F * math.exp(-r * T) * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'Put':
        option_price = K * math.exp(-r * T) * norm.cdf(-d2) - F * math.exp(-r * T) * norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'Call' or 'Put'.")
    
    return option_price

For static replication of any constant maturity swap (CMS) payoff $g(F)$, where $F$ is the swap rate, we use the following formula:

  \begin{equation*}
    \begin{split}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

where

  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\mbox{IRR}(K)} \\
      h'(K) &= \frac{\mbox{IRR}(K)g'(K) - g(K)\mbox{IRR}'(K)}{\mbox{IRR}(K)^2} \\
      h''(K) &= \frac{\mbox{IRR}(K)g''(K)-\mbox{IRR}''(K)g(K) -2\cdot\mbox{IRR}'(K)g'(K)}{\mbox{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\mbox{IRR}'(K)^2g(K)}{\mbox{IRR}(K)^3}.
    \end{split}
  \end{equation*}
  
For CMS rate payoff, the payoff function can be defined simply as $g(F)=F$, and the static replication formula simplifies into:

  \begin{equation*}
    \begin{split}
      D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

We can implement this in Python. First we define the IRR functions.

Let $m$ denote the payment frequenc ($m=2$ for semi-annual payment frequency), and let $N = T_N-T_n$ denote the tenor of the swap (number of years), the partial derivatives on the IRR function $\mbox{IRR}(S)$ given by:
\begin{equation*}
\begin{split}
\mbox{IRR}(K)&=\sum_{i=1}^{N\times m}\frac{1}{(1+\frac{K}{m})^i}=\frac{1}{K}\left[1-\frac{1}{\left(1+\frac{K}{m}\right)^{N\times m}}\right]\\
\mbox{IRR}'(K)&=-\frac{1}{K}\mbox{IRR}(K)
+\frac{1}{m\times K}\frac{N\times m}{\left(1+\frac{K}{m}\right)^{N\times m+1}} \\
\mbox{IRR}''(K)&=-\frac{2}{K}\mbox{IRR}'(K)
-\frac{1}{m^2\times K}\frac{N\times m\cdot (N\times m+1)}{\left(1+\frac{K}{m}\right)^{N\times m+2}} \\
\end{split}
\end{equation*}

These results will need to be generalised to handle the case for $m=2$ to be consistent with the semi-annual payment frequency swap market data provided.


In [9]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative

## Question 1

For this CMS rate payment, since $g(F)=$$F^{\frac{1}{4}}$$-0.2$, we have the derivatives:

\begin{equation*}
\begin{split}
g(K) &= K^{\frac{1}{4}}-0.2 \\
g'(K) &= {\frac{1}{4}}K^{-\frac{3}{4}} \\
g''(K) &= {-\frac{3}{16}}K^{-\frac{7}{4}} \\
\end{split}
\end{equation*}

In [10]:
def g_0(K):
    return K**(1/4) -0.2

def g_1(K):
    return (1/4)*K**(-3/4)

def g_2(K):
    return -(3/16)*K**(-7/4)

  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\mbox{IRR}(K)} \\
      h'(K) &= \frac{\mbox{IRR}(K)g'(K) - g(K)\mbox{IRR}'(K)}{\mbox{IRR}(K)^2} \\
      h''(K) &= \frac{\mbox{IRR}(K)g''(K)-\mbox{IRR}''(K)g(K) -2\cdot\mbox{IRR}'(K)g'(K)}{\mbox{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\mbox{IRR}'(K)^2g(K)}{\mbox{IRR}(K)^3}.
    \end{split}
  \end{equation*}

In [11]:
def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / (IRR_0(K, m, N)**2)
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

We will also need to implement the IRR-settled payer and receiver swaption formulae:

  \begin{equation*}
    \begin{split}
      V^{pay}_{n,N}(0) &= D(0,T_n) \cdot \mbox{IRR}(S_{n,N}(0)) \cdot \mbox{Black76Call}(S_{n,N}(0),K,\sigma_{n,N},T) \\
      V^{rec}_{n,N}(0) &= D(0,T_n) \cdot \mbox{IRR}(S_{n,N}(0)) \cdot \mbox{Black76Put}(S_{n,N}(0),K,\sigma_{n,N},T) \\
    \end{split}
  \end{equation*}

where $S_{n,N}(0)=F$ is today's forward swap rate calculated based on the curves we bootstrapped, and $\sigma_{n,N}$ is the SABR implied volatility calibrated to swaption market data.

### Value the payoff

  \begin{equation*}
    \begin{split}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

In [12]:
# To get sigma for Black76, we need repvious SABR
F = data_FS.iloc[9,2] # Forward Swap from part 1(5 years Expiry, 10 years tenor)
T = 5
# SABR(F, K, T, alpha, 0.9, rho, nu)

DT = data_DF.iloc[9,3] # Discount factor (5 years Expiry, 10 years tenor)
m = 2 # Semi-annual 
N = 10 # Tenor

V_pay = integrate.quad(lambda k: h_2(k, m, N)*black76_lognormal('Call', F, k, 0, SABR(F, k, T, alpha, 0.9, rho, nu), T), F, 1000)
V_rec = integrate.quad(lambda k: h_2(k, m, N)*black76_lognormal('Put', F, k, 0, SABR(F, k, T, alpha, 0.9, rho, nu), T), 0, F)

V_0 = DT * g_0(F) + V_rec[0] + V_pay[0] # The second term should be zero
print("Question 1 Payoff:", V_0)

Question 1 Payoff: 0.24986290267550515


## Question 2

  \begin{equation*}
    \begin{split}
      Caplet &= h'(X)V^{pay}(X) 
      &\;+ 
      \int_X^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

In [13]:
# F**(1/4) > 0.2, F > 0.0016
X = 0.0016

In [14]:
V_pay = integrate.quad(lambda k: h_2(k, m, N)*black76_lognormal('Call', F, k, 0, SABR(F, k, T, alpha, 0.9, rho, nu), T), X, 1000)
h_1_term = h_1(X, m, N)*black76_lognormal('Call', F, X, 0, SABR(F, X, T, alpha, 0.9, rho, nu), T)
                                          
V_caplet = h_1_term + V_pay[0]
print("Question 2 Payoff:", V_caplet)

Question 2 Payoff: 0.031030476391866207
